*********************************************************************************************************
# A Tour of Python 3  
version 1.0.1  
Authors: Phil Pfeiffer, Zack Bunch, and Feyisayo Oyeniyi  
East Tennessee State University  
Last updated June 2021  
*********************************************************************************************************

# 10.  Classes  
 10.1 [Overview](#Classes-Overview)  
 10.2 [Basic class properties](#Classes-Basic-Properties)  
 10.3 [Methods](#Classes-Methods)  
 &ensp; 10.3.1 [Instance methods](#Classes-Instance-Methods)  
 &ensp; 10.3.2 [Class methods](#Classes-Class-Methods)  
 &ensp; 10.3.3 [Static Methods](#Classes-Static-Methods)  
 10.4 [Method invocation within a class](#Classes-Method-Invocation-Within-Classes)  
 10.5 [Operator customization](#Classes-Operator-Customization)  
 &ensp; 10.5.1 [Iterators](#Classes-Operator-Customization-Iterators)  
 &ensp; 10.5.2 [Serialization, via `__repr__`](#Classes-Operator-Customization-Repr)  
 &ensp; 10.5.3 [Relational operators](#Classes-Operator-Customization-Relational-Operators)  
 &ensp; 10.5.4 [Attribute management](#Classes-Operator-Customization-Attribute-Management)  
 10.6 [Properties](#Classes-Properties)   
 10.7 [Managing polymorphism](#Classes-Managing-Polymorphism)   
 10.8 [ Multiple inheritance](#Classes-Multiple-Inheritance)  
 &ensp; 10.8.1 [About multiple inheritance](#Classes-Multiple-Inheritance-About)   
 &ensp; 10.8.2 [Multiple inheritance in Python](#Classes-Multiple-Inheritance-In-Python)  
 &ensp; 10.8.3 [Accessing methods hidden by other methods](#Classes-Multiple-Inheritance-Accessing_Hidden-Methods)

## 10.1  Overview <a name='Classes-Overview'></a>

A Python class can be regarded as a *unique* *container* whose *content* *can be used* to *generate* a *family* of *objects*:

-  *unique*.  Each class has an identity that differentiates it from all other objects in Python's environment.
-  *container*.  A Python class is structured as a list of references to values.   These references are referred to as a class's **attributes**.
-  *content*.  Python classes act as namespaces.  Their attributes are accessed with expressions that, directly or indirectly,    reference their classes by name.
-  *can be used*.  Python statements and operators commonly work by invoking "special" attributes with well-known names:  e.g.,
    -  Python's `dir` built-in invokes a class's `__dir__` attribute to list that class's attributes.
    -  Python's `id` built-in invokes class's `__hash__` attribute to retrieve that class's unique identifier.
-  *generate*.  Python's constructor mechanism invokes two special attributes,    `__new__ `and `__init__`, to create and initialize a new object.
-  *family*.  Python treats a class as a type.  All objects generated from a class initially have that class as their type. 
     (Note: a code can, subject to certain restrictions, dynamically change an object's type.)
-  *object*.  An object, according to Meiler Page-Jones, is an entity that has 
      [an identity, a state, an interface, behaviors, and a lifetime](./3.%20%20Objects%20and%20Identifiers.ipynb).

In addition to these five characteristics, Python 3 objects are typed and subclassed.
-  Every class is a subclass of at least one other class: possibly, of two or more.
-  Ultimately, all classes are descended from `object`, the root of Python's class hierarchy.

As in other object-oriented (OO) languages, saying that a class B is a subclass of a class A  means B can *inherit* A's content:  i.e., access and manipulate A's content directly, as if that content were part of B's explicit definition.

## 10.2  Basic class properties <a name='Classes-Basic-Properties'></a>

Classes are created by an executable statement, `class`.  This statement consists of two parts:
-  a header that names the class to create, and, optionally, its superclasses  (default: `object`)
-  a body that specifies the class's initial content

In [ ]:
# 10.2.a  creating a trivial class and exploring its attributes

class Trivial:  pass    # as simple as one can get

print( 'Trivial\'s id is', id(Trivial) )      # all objects have unique ids
print( 'Trivial\'s base class is', Trivial.__class__.__base__  )    # returns a class's immediate superclass

attributes_only_in_base = set(dir(Trivial.__class__.__base__)) - set(dir(Trivial))
only_in_base = attributes_only_in_base if attributes_only_in_base else 'None'
print( f'Attributes in {Trivial.__class__.__base__} missing from Trivial: {only_in_base}' )

attributes_only_in_Trivial = set(dir(Trivial)) - set(dir(Trivial.__class__.__base__))
only_in_Trivial = attributes_only_in_Trivial if attributes_only_in_Trivial else 'None'
print( f'Attributes in Trivial missing from {Trivial.__class__.__base__}: {only_in_Trivial}' )

In [ ]:
# 10.2.b  working with a Trivial class's derived classes

class Trivial:  pass    # as simple as one can get

instance_1 = Trivial()
instance_2 = Trivial()

print( f"instance_1 is{'' if isinstance( instance_1, Trivial ) else ' not'} an instance of Trivial" )
print( f"instance_2 is{'' if isinstance( instance_2, Trivial ) else ' not'} an instance of Trivial" )

print( f"instance_1 is {'the same as' if id( instance_1 ) == id( Trivial ) else 'distinct from'} class Trivial" )
print( f"instance_2 is {'the same as' if id( instance_2 ) == id( Trivial ) else 'distinct from'} class Trivial" )
print( f"instance_1 is {'the same as' if id( instance_1 ) == id( instance_2 ) else 'distinct from'} instance_2" )

## 10.3  Methods <a name='Classes-Methods'></a>

When a class constructs an instance of itself, it associates that instance with a directory of references. This directory includes two kinds of references:
-  references to entities that are local to the instance
   -  These references are to data items that belong to the instance proper,       including attributes that are added to the instance as execution proceeds.
   -  By convention, these local references are accessed via a *namespace* known as `self`.
-  references to entities that all instances of a class share.  These include
   -  all code objects that correspond to the methods that a class defines
   -  any data items that are local to the class definition

A class's methods, in turn, can be divided into three basic categories, according to what they can access.
-  *instance* methods.  Instance methods access data associated with individual instances of classes.
-  *class* methods.  class methods access data associated with the class as a whole:  e.g., class-related defaults. 
-  *static* methods.  static methods, as a rule, implement actions associated with a class's purpose, but not with its      mutable data: e.g., converting between inches and centimeters or cartesian and polar coordinates.

This admittedly subtle distinction between class methods and static methods, if accounted for in a class's design, 
can help to clarify that design's intent.

### 10.3.1  Instance methods <a name='Classes-Instance-Methods'></a>

Instance methods include a special first parameter, typically called `self`, that references the instance's private data. All instance data should be stored in and accessed from `self`.

In [ ]:
# 10.3.1  a class with only instance methods

class MyClass:
  def set_value(self, v): 
    self.value = v
  def get_value(self): 
    return self.value

# create a trivial subclass of this class to illustrate the methods' operation

class MySubclass(MyClass): pass

# create class instances to work with

myclass_instance_1 = MyClass()
myclass_instance_2 = MyClass()
mysubclass_instance = MySubclass()

# - - - instance methods are commonly invoked by class instance - - -

print( 'setting instance 1, instance 2, and subclass instance value ', end='')
print( 'to 1, 2, and 3, respectively, in succession' )
myclass_instance_1.set_value(1)
myclass_instance_2.set_value(2)
mysubclass_instance.set_value(3)

print( 'my class instance 1\'s value is now', myclass_instance_1.get_value() )
print( 'my class instance 2\'s value is now', myclass_instance_2.get_value() )
print( 'my subclass class instance\'s value is now', mysubclass_instance.get_value() )
print()

# - - - instance methods can also be invoked by class name, if supplied with an instance of that class - - -

print( 'setting instance 1, instance 2, and subclass instance value,', end='')
print( 'to 4, 5, and 6, respectively, in succession' )
MyClass.set_value(myclass_instance_1, 4)
MyClass.set_value(myclass_instance_2, 5)
MySubclass.set_value(mysubclass_instance, 6)

print( 'my class instance 1\'s value is', myclass_instance_1.get_value() )
print( 'my class instance 2\'s value is', myclass_instance_2.get_value() )
print( 'my subclass class instance\'s value is', mysubclass_instance.get_value() )

### 10.3.2  Class methods <a name='Classes-Class-Methods'></a>

Class methods are defined with a special first parameter: the class that 'owns' the attributes that the method references. These methods must be wrapped in logic that preprocesses their inputs. This wrapping can be done in one of two ways:
- by setting the method's name to the output of a built-in Python functional, `classmethod`.
- by prepending Python's `@classmethod` [decorator](./7.%20%20Functions.ipynb#Functions-Decorator) to the function's definition.

In [ ]:
# 10.3.2.a a class with only class methods

class MyClass:
  def set_classattr_foo(thisclass, value):            # these 3 lines define a classmethod for MyClass
    thisclass.foo = value
  set_classattr_foo = classmethod(set_classattr_foo)  # required to make set_classattr a classmethod
  def get_classattr_foo(thisclass):                   # these 3 lines define a classmethod for MyClass
    return thisclass.foo
  get_classattr_foo = classmethod(get_classattr_foo)  # required to make get_classattrs a classmethod

# create a trivial subclass of this class to illustrate the methods' operation

class MySubclass(MyClass): pass

# create class instances to work with

myclass_instance_1 = MyClass()
myclass_instance_2 = MyClass()
mysubclass_instance = MySubclass()

# - - - classmethods can be invoked by class instance

print( 'setting class attribute referenced by instance 1, instance 2, ', end='' )
print( 'and subclass instance to 1, 2, and 3, respectively, in succession' )
myclass_instance_1.set_classattr_foo(1)
myclass_instance_2.set_classattr_foo(2)
mysubclass_instance.set_classattr_foo(3)

print( 'my class instance 1\'s value is now', myclass_instance_1.get_classattr_foo() )
print( 'my class instance 2\'s value is now', myclass_instance_2.get_classattr_foo() )
print( 'my subclass class instance\'s value is now', mysubclass_instance.get_classattr_foo() )

# - - - classmethods are more commonly invoked directly, by class name; no instance is required - - -

print()
print( 'setting class attribute referenced by instance 1, instance 2, and subclass instance ', end='')
print( 'to 4 and 5, respectively, in succession' )
MyClass.set_classattr_foo(4)
MySubclass.set_classattr_foo(5)

print( 'my class instances\' value is now', MyClass.get_classattr_foo() )
print( 'my subclass class instance\'s value is now', MySubclass.get_classattr_foo() )

In [ ]:
# 10.3.2.b   using the classmethod decorator to instantiate class methods

class MyClass:
  @classmethod
  def set_classattr_foo(thisclass, value):  thisclass.foo = value
  @classmethod
  def get_classattr_foo(thisclass):  return thisclass.foo

# invoke the setter, then the getter

MyClass.set_classattr_foo(12)
print( 'my class instance\'s value is', MyClass.get_classattr_foo() )

In [ ]:
# 10.3.2.c  initializing the class method's supporting attribute

class MyClass:
  foo = '<initial value>'
  @classmethod
  def set_classattr_foo(thisclass, value):  thisclass.foo = value
  @classmethod
  def get_classattr_foo(thisclass):  return thisclass.foo

# invoke the getter, then the setter, then the getter

print( 'my class instance\'s value initial value is', MyClass.get_classattr_foo() )
MyClass.set_classattr_foo(12)
print( 'my class instance\'s value after update is', MyClass.get_classattr_foo() )

### 10.3.3  Static methods <a name='Classes-Static-Methods'></a>

Static methods are defined without a special first parameter. These methods must be wrapped in logic that preprocesses their inputs. This wrapping can be done in one of two ways:
- by setting the method's name to the output of a built-in Python functional, `staticmethod`.
- by prepending Python's `@staticmethod` [decorator](./7.%20%20Functions.ipynb#Functions-Decorator) to the function's definition.

In [ ]:
# 10.3.3.a  a class with only static methods

class MyClass:
  def set_static_value(newv):                         # these 2 lines define a setter for MyClass's static_value variable
    MyClass.static_value = newv
  set_static_value = staticmethod(set_static_value)   # this line redefines set_static_value as a staticmethod
  def get_static_value():                             # these 2 lines define a getter for MyClass's static_value variable
    return MyClass.static_value
  get_static_value = staticmethod(get_static_value)   # this line redefines get_static_value as a staticmethod

# create a trivial subclass of this class to illustrate the methods' operation

class MySubclass(MyClass): pass

# create class instances to work with

myclass_instance_1 = MyClass()
myclass_instance_2 = MyClass()
mysubclass_instance = MySubclass()

# - - - staticmethods can be invoked by class instance

print( 'setting static value referenced by instance 1, instance 2, and subclass instance ', end='')
print( 'to 1, 2, and 3, respectively, in succession' )
myclass_instance_1.set_static_value(1)
myclass_instance_2.set_static_value(2)
mysubclass_instance.set_static_value(3)


print( 'my class instance 1\'s static value is', myclass_instance_1.get_static_value() )
print( 'my class instance 2\'s static value is', myclass_instance_2.get_static_value() )
print( 'my subclass class instance\'s static value is', mysubclass_instance.get_static_value() )
print()

# - - - staticmethods are more commonly invoked directly, by class name; no instance is required - - -

print( 'setting static value referenced by MyClass and MySubclass to 4 and 5, ', end='')
print( 'respectively, in succession' )
MyClass.set_static_value(4)
MySubclass.set_static_value(5)

print( 'my class instances\' static value is', MyClass.get_static_value() )
print( 'my subclass class instance\'s static value is', MySubclass.get_static_value() )

In [ ]:
# 10.3.3.b   using the staticmethod decorator to instantiate static methods

class MyClass:
  @staticmethod
  def set_static_value(newv):   MyClass.static_value = newv
  @staticmethod
  def get_static_value():       return MyClass.static_value

# invoke the setter, then the getter

MyClass.set_static_value(10)
print( 'my class\'s static value is', MyClass.get_static_value() )

In [ ]:
# 10.3.3.c  initializing a static value as part of class definition

class MyClass:
  static_value = '<initial value>'
  @staticmethod
  def set_static_value(newv):   MyClass.static_value = newv
  @staticmethod
  def get_static_value():       return MyClass.static_value

# invoke the getter, then the setter, then the getter

print( 'my class\'s initial static value is', MyClass.get_static_value() )
MyClass.set_static_value(10)
print( 'my class\'s static value was updated to', MyClass.get_static_value() )

## 10.4  Method invocation within a class  <a name='Classes-Method-Invocation-Within-Classes'></a>

To call a method from inside a class, prefix the method with either
-  `self`, for instance and static methods
-  a class name plus `self` as a first argument, for instance and static methods
-  an identifier that names a class, for class methods


In [ ]:
# 10.4.1.a  invoking an instance method from within a class, using self

class MyClass:
  def __init__(self, v):     # Python's dedicated name for class constructors
    self.set_value(v)
  def set_value(self, v): 
    self.value = v
  def get_value(self): 
    return self.value

# create a class instance to work with, then invoke the getter
myclass_instance = MyClass(4)
print( 'my class instance\'s value is', myclass_instance.get_value() )

In [ ]:
# 10.4.1.b  invoking an instance method from within a class, using the class name

class MyClass:
  def __init__(self, v):     # Python's dedicated name for class constructors
    MyClass.set_value(self,v)
  def set_value(self, v): 
    self.value = v
  def get_value(self): 
    return self.value

# create a class instance to work with, then invoke the getter
myclass_instance = MyClass(4)
print( 'my class instance\'s value is', myclass_instance.get_value() )

In [ ]:
# 10.4.1.c  invoking a static method from within a class

class MyClass:
  static_value = 'initial value'
  def __init__(self, v):     # Python's dedicated name for class constructors
    self.set_static_value(v)
  @staticmethod
  def set_static_value(newv):   MyClass.static_value = newv
  @staticmethod
  def get_static_value():       return MyClass.static_value

# invoke the getter, create two class instances to work with, then invoke the getters

print( 'MyClass\'s initial value is', MyClass.get_static_value() )
print( 'updating instance_1 and instance_2 to 4 and 5, respectively, in succession' )
myclass_instance_1 = MyClass(4)
myclass_instance_2 = MyClass(5)
print( 'my class instance 1\'s value after updates is', myclass_instance_1.get_static_value() )
print( 'my class instance 2\'s value after updates is', myclass_instance_2.get_static_value() )

In [ ]:
# 10.4.1.d  invoking a class method from within a class

class MyClass:
  my_attribute = 'initial value'
  def __init__(self, v):     # Python's dedicated name for class constructors
    MyClass.set_my_attribute(v)
  @classmethod
  def set_my_attribute(thisclass, newv):   thisclass.my_attribute = newv
  @classmethod
  def get_my_attribute(thisclass):         return thisclass.my_attribute

# invoke the getter, create two class instances to work with, then invoke the getters

print( 'MyClass\'s initial value is', MyClass.get_my_attribute() )
print( 'MyClass\'s initial value is', MyClass.get_my_attribute() )
print( 'updating instance_1 and instance_2 to 4 and 5, respectively, in succession' )
myclass_instance_1 = MyClass(4)
myclass_instance_2 = MyClass(5)
print( 'my class instance 1\'s value is', myclass_instance_1.get_my_attribute() )
print( 'my class instance 2\'s value is', myclass_instance_2.get_my_attribute() )

## 10.5  Operator customization  <a name='Classes-Operator-Customization'></a>

### 10.5.1  Iterators  <a name='Classes-Operator-Customization-Iterators'></a>

Iterators use `__iter__` methods to determine the sequence of values that a class instance returns when included in a loop. Two idioms are commonly used to implement an iterator. One, shown in the next example, structures `__iter__` as a self-contained generator for returning a desired sequence.

In [ ]:
# 10.5.1.a  the earlier fib function, recast as a class with an __iter__ method

class Fib:
  def __init__(self, val_count = float('inf')):
    self.val_count = val_count
  def __iter__(self):
    if self.val_count < 1: return
    yield 1
    self.val_count -= 1
    if self.val_count < 2: return
    yield 1
    self.two_ago, self.one_ago = 1, 1
    self.val_count -= 2
    while self.val_count > 0:
      self.val_count -= 1
      next_to_yield = self.two_ago + self.one_ago
      yield next_to_yield
      self.two_ago, self.one_ago = self.one_ago, next_to_yield

fibgen = Fib(10)

# the following works once, but not twice, since val_count is never reset
print( 'first  pass through fibgen: ', [i for i in fibgen] )
print( 'second pass through fibgen: ', [i for i in fibgen] )

<span style='color:blue'>&#128073;&ensp;&ensp;**Exercise 10.5.1.1:**

</span><span style='color:navy'>The Taylor series approximation to &#120587; is given by the formula 4 &ast; (1/1 - 1/3 + 1/5 - 1/7 + 1/9 - 1/11 ...). In the following code cell, create a class with an `__init__` and an `__iter__` function, similar to the above, that, when invoked for the kth time, returns 4 times the sum of the first k values in this sequence, along with examples that demonstrate its operation.</span>

 The other, more common idiom treats `__iter__` as an initializer for a second special attribute, `__next__`.  In this second idiom, shown below,
-  `__iter__` is defined as a two-part method that
    -  initializes any state that's needed to support the iteration, then
    -  returns `self`, in order to hand off the computation to `__next__`.
-  `__next__` then uses the class's state to return the required sequence of elements.

In [ ]:
# 10.5.1.b  the earlier fib function, recast as a class with __iter__ and __next__ methods

class Fib:
  def __init__(self, val_count = float('inf')):
    self.initial_val_count = val_count
    self.init_iter()
  def __iter__(self):
    self.init_iter()
    return self
  def __next__(self):
    if self.val_count <= 0: raise StopIteration
    self.val_count -= 1
    next_result = self.two_ago + self.one_ago
    self.two_ago, self.one_ago = self.one_ago, next_result
    return next_result
  def init_iter(self):
    self.val_count, self.two_ago, self.one_ago = self.initial_val_count, 0, 1

fibgen = Fib(10)

# the following works twice, since __iter__ resets val_count
print( 'first  pass through fibgen: ', [i for i in fibgen] )
print( 'second pass through fibgen: ', [i for i in fibgen] )

<span style='color:blue'>&#128073;&ensp;&ensp;**Exercise 10.5.1.2:**

</span><span style='color:navy'>In the following code cell, repeat the Taylor series example, but with a class with an `__init__` , an `__iter__`, and a `__next__` function, similar to the above.</span>

### 10.5.2  Serialization via `__repr__`  <a name='Classes-Operator-Customization-Repr'></a>

*Serialization* is the saving of an object's state for recall at a later time.  Common reasons for serializing objects include
-  Dispatching part of a computation to another host, to improve its efficiency or fault-tolerance
-  Checkpointing a long-running computation's state, to aid in failure recovery.
-  Examining a computation's intermediate states, in order to
    -  verify that program's correctness or
    -  debug that program's execution
-  Using a first computation's output as the input to a second.

Various third-party packages have been developed for serializing Python object content in relational databases. These packages are known as *object-relational mappers*, or ORMs for short.  One such ORM for Python, [SQLAlchemy](https://www.sqlalchemy.org), supports a variety of popular back-end databases, including MySQL, PostgreSQL, and SQLite. 

The Python distribution provides two lighter-weight mechanisms for serializing object content. One, *pickling*, is supported by the Python library's [pickle module](https://docs.python.org/3/library/pickle.html). The other, Python's `repr` built-in, uses a special, class-specific method, `__repr__`.  If a class meets the following three conditions--
-  it has a constructor that can be used to populate all of its member data
-  it contains no opaque member data like code
-  its `__eq__` method tests for equivalence based on common type and common state

\-\-then `repr` can be defined to output a string that, when supplied to `eval`, recreates the serialized instance.

The following example uses this additional Python construct:
-  `__class__.__name__` - The class attribute that references a class's name

In [ ]:
# 10.5.2.a  showing the use of __repr__ to serialize and restore a class instance's state

class MyClass:
  # important - the class has a constructor that can populate all of its state
  def __init__(self, iv_1, iv_2):  self.iv_1, self.iv_2 = iv_1, iv_2
  #
  # important - the class has a repr method that restores all of its state
  def __repr__(self):  return f'{self.__class__.__name__}({self.iv_1!r},{self.iv_2!r})'
  #
  # important - __eq__ checks for state equivalence, while __ne__ inverts the __eq__ check
  def __eq__(self, other):  return isinstance(other, MyClass) and self.iv_1 == other.iv_1 and self.iv_2 == other.iv_2
  def __ne__(self, other):  return not(self.__eq__(other))

x = MyClass([1, 2], {'three':3})

print( 'repr(x), a.k.a. MyClass([1, 2], {\'three\':3}).repr(), is', repr(x) )
print( f"x {'equals' if eval(repr(x)) == x else 'does not equal'} eval(repr(x))" )

In [ ]:
# 10.5.2.b  repr-izing lambdas, however, doesn't really work

make_printable = lambda exception: '' if str(exception) is None else str(exception)

f = lambda: 3
print( 'repr(f), a.k.a. lambda: 3, is', repr(f) )
try:
  eval_repr_f =  eval(repr(f))
  print( f"f {'equals' if f == eval(repr(f)) else 'does not equal'} eval(repr(f))" )
except Exception as exception:
  print( 'attempt to execute eval(repr(f)) failed:', make_printable(exception) )

### 10.5.3  Relational operators  <a name='Classes-Operator-Customization-Relational-Operators'>

Each of Python's relational operators implements its comparison by invoking a special attribute associated with its left-hand operand:
-  &lt;  invokes `__lt__` 
-  &le; invokes `__le__` 
-  &ge; invokes `__ge__` 
-  &gt; invokes `__gt__` 
-  == invokes `__eq__` 
-  != invokes `__ne__` 

These operations can be tailored to a class's semantics by redefining them in a class's definition.  Python requires these attributes to be defined as functions of two arguments:
-  `self`,  the current object's state
-  `other`, the state of the relational operator's right-hand operand.

Some groups of objects have no sensible ordering. Attempts to compare incomparable objects should return `NotImplemented`. Python converts `NotImplemented` to `False` in the absence of a check for `NotImplemented`.

Every user-defined class **should** have appropriate definitions for `__eq__` and `__ne__`.  This is of particular concern for `object`'s immediate subclasses, since `object.__eq__` and `object.__ne__` use `id` to test for equality:  i.e.,

&ensp;&ensp;&ensp;&ensp; object_a == object_b iff they reference the same (memory) object.

`id` is inappropriate for more typical comparisons that assess behavior and state:  i.e.,

&ensp;&ensp;&ensp;&ensp; object_a == object_b  iff they are of the same type and have the same state.

Unlike languages like Ruby that do one-way, "right to left" tests for equality, Python's == operator is apparently symmetric:  i.e.,

&ensp;&ensp;&ensp;&ensp; a == b iff  a.*__*
eq__(b) and b.*__*eq__(a)

This property, however, does not imply that == is transitive.

The following examples use this additional Python construct:
-  `super().__init__( )` - invokes the `__init__` method of a class's immediate superclass(es).

In [ ]:
# 10.5.3.a  list with "existential" comparison operators
# The following definition of relational operations is one that the XSLT programming language uses for comparing lists.
#  Essentially, it holds that a <compare> b is true iff a[i] <compare> b[j] for some a[i] in a and b[j] in b

class XSLT_list(list):
  def __lt__(self, other):
    return NotImplemented if not isinstance(other, XSLT_list) else any([a <  b for a in self for b in other])
  def __le__(self, other):
    return NotImplemented if not isinstance(other, XSLT_list) else any([a <= b for a in self for b in other])
  def __eq__(self, other):
    return isinstance(other, XSLT_list) and any([a == b for a in self for b in other])
  def __ne__(self, other):
    return not isinstance(other, XSLT_list) or any([a != b for a in self for b in other])
  def __ge__(self, other):
    return NotImplemented if not isinstance(other, XSLT_list) else any([a >= b for a in self for b in other])
  def __gt__(self, other):
    return NotImplemented if not isinstance(other, XSLT_list) else any([a >  b for a in self for b in other])

list0 = XSLT_list([])
list1 = XSLT_list([0, 1, 2])
list2 = XSLT_list([2, 3, 4])
list3 = XSLT_list([4, 5, 6])

for list_a in [list0, list1, list2, list3]:
  for list_b in [list0, list1, list2, list3]:
    print( f'{list_a} <  {list_b} is {list_a <  list_b}' )
    print( f'{list_a} <= {list_b} is {list_a <= list_b}' )
    print( f'{list_a} == {list_b} is {list_a == list_b}' )
    print( f'{list_a} != {list_b} is {list_a != list_b}' )
    print( f'{list_a} >= {list_b} is {list_a >= list_b}' )
    print( f'{list_a} >  {list_b} is {list_a >  list_b}' )
    print( )

In [ ]:
# 10.5.3.b  extend list's built-in ordering relations with an ownership check

class MyList(list):
  def __init__(self, l, owner ):
    super().__init__( l )
    self.owner = owner
  #
  def __eq__(self, other):
    return isinstance(other, MyList) and super().__eq__(other) and self.owner == other.owner
  def __ne__(self, other):
    return not isinstance(other, MyList) or super().__ne__(other) or  self.owner != other.owner
  def __str__(self):
    return  str((super().__str__(), self.owner))

mylist_123_phil = MyList([1, 2, 3], 'Phil')
mylist_456_phil = MyList([4, 5, 6], 'Phil')
mylist_123_bob  = MyList([1, 2, 3], 'Bob')

for list_a in [mylist_123_phil, mylist_456_phil, mylist_123_bob]:
  for list_b in [mylist_123_phil, mylist_456_phil, mylist_123_bob]:
    print( f'{list_a} == {list_b} is {list_a == list_b}' )
    print( f'{list_a} != {list_b} is {list_a != list_b}' )
    print( )

<span style='color:blue'>&#128073;&ensp;&ensp;**Exercise 10.5.3.1:**

</span><span style='color:navy'>In the following markdown cell, explain the reason for calling `super().__str__` in MyList's `__str__` routine.</span>
***


***


<span style='color:blue'>&#128073;&ensp;&ensp;**Exercise 10.5.3.2:**

</span><span style='color:navy'>In the following markdown cell, describe what changes, if anything, if the call to MyList's `__str__` routine is deleted, accounting for any such changes.</span>
***


***


<span style='color:blue'>&#128073;&ensp;&ensp;**Exercise 10.5.3.3:**

</span><span style='color:navy'>In the following markdown cell, describe what changes, if anything, if the call to `str()` in MyList's `__str__` routine is removed, leaving just the tuple.</span>
***


***


<span style='color:blue'>&#128073;&ensp;&ensp;**Exercise 10.5.3.4:**

</span><span style='color:navy'>In the following code cell,  create and demonstrate the use of a `__repr__` method for MyList. Define this method so that `eval(repr(m)) == m` for any instance m of MyList.</span>

### 10.5.4  Attribute management  <a name='Classes-Operator-Customization-Attribute-Management'></a>

Python provides two built-in methods for retrieving and updating an object's attributes.  With a few exceptions involving special attributes like `__hash__`, access to an object's attributes can be controlled by recoding
-  `__getattribute__`, which dereferences identifiers in contexts where their values are read
-  `__setattr__`, which dereferences identifiers in contexts where their values are updated

In [ ]:
# 10.5.4.a   overloading of __getattribute__ and __setattr__  to return information on faculty offices

class DeptOffices:
  # ETSU Dept. of Computing offices as of Spring 2021
  nicks = 'Nicks'
  mlctr = 'Millennium Center'
  ww =    'Wilson-Wallis'
  all_dept_offices = \
     {('Don', 'Bailes'): (459, nicks),     ('Brian', 'Bennett'): (463, nicks),    ('Erin', 'Coker'): (None, None), 
      ('Erin', 'Cook'): (470, nicks),      ('Corey', 'Dean'): (153, mlctr),       ('Mathew', 'Desjardins'): (151, mlctr),
      ('Chelsie', 'Dubay'): (466, nicks),  ('Esra', 'Erdin'): (474, nicks),       ('Jeff', 'Fraley'): (477, nicks),  
      ('Ed', 'Hall'): (155, mlctr),        ('Matt', 'Harrison'): (157, mlctr),    ('Stephen', 'Hendrix'): (468, nicks), 
      ('Ghaith', 'Husari'): (460, nicks),  ('Ferdaus', 'Kawsar'): (486, nicks),   ('Mohammad', 'Khan'): (457, nicks),  
      ('Ken', 'Loveday'): (112, mlctr),    ('Robert', 'Nielsen'): (475, nicks),   ('Phil', 'Pfeiffer'): (467, nicks),  
      ('Tony', 'Pittarese'): (464, nicks), ('Jack', 'Ramsey'): (484, nicks),      ('Tahsin', 'Rezwana'): (479, nicks), 
      ('Jeff', 'Roach'): (473, nicks),     ('William', 'Rochelle'): (487, nicks), ('David', 'Robinson'): ('6B', ww), 
      ('David', 'Tarnoff'): (469, nicks),  ('Chris', 'Wallace'): (154, mlctr) 
     } 
  #
  def __init__(self, building):
    self.this_building = building
  #
  def __getattribute__(self, attr_name):
    if any(attr_name in person_name for person_name in DeptOffices.all_dept_offices.keys()):
      print( '> accessing virtual class attribute DeptOffices.' + attr_name )
      return [(name, (room, building)) for (name, (room, building)) in DeptOffices.all_dept_offices.items() \
                   if attr_name in name and self.this_building == building]
    else:
      return super().__getattribute__(attr_name)

nicks_offices, millennium_offices = DeptOffices( 'Nicks' ), DeptOffices( 'Millennium Center' )

for ( offices_in, building_name ) in [ ( 'nicks_offices', 'Department'), ( 'millennium_offices', 'Millennium Center' ) ]:
  for person_name in ['Phil', 'David', 'Erin', 'Ed', 'Fred']:
    try:
      office_list = eval( offices_in + '.' + person_name )
      if office_list:
        print( f'{building_name} offices for people named {person_name} include {office_list}' )
      else:
        print( f'No {building_name} offices for people named {person_name}' )
    except:
      print( f'{offices_in} lacks an attribute for {person_name}' )
  print()

<span style='color:blue'>&#128073;&ensp;&ensp;**Exercise 10.5.4.1:**

</span><span style='color:navy'>In the following code cell,  extend DeptOffices's `__init__` method so that it also provides a way of finding someone's office, regardless of building.  Illustrate the extended class's operation with a set of examples. Suggestion: `None` is a common means of specifying a default action, including a more general one.</span>

In [ ]:
# 10.5.4.b   overloading of __getattribute__ and __setattr__ to disable direct access to a variable, w

class MyClass(object):
  # since redefinition of MyClass.__getattribute__ and MyClass.__setattr__ thwarts attempts to access w,
  # __init___, get_w, and set_w must manipulate w using the superclass's __setattr__ and __getattribute__
  #
  def __init__( self, v, w ):
    self.set_v(v)
    super().__setattr__( 'w', w )

  def get_v( self ):     return super().__getattribute__( 'v' )
  def set_v( self, v ):  return super().__setattr__( 'v', v )

  def get_w( self ):     return super().__getattribute__( 'w' )
  def set_w( self, w ):  return super().__setattr__( 'w', w )

  def __getattribute__(self, attr_name):
    if attr_name == 'w':  raise LookupError("w is private -- can't be read")
    return super().__getattribute__(attr_name)
  def __setattr__(self, attr_name, x):
    if attr_name == 'w':  raise LookupError("w is private -- can't be written")
    return super().__setattr__(attr_name, x)

make_printable = lambda exception: '' if str(exception) is None else str(exception)

instance = MyClass(1, 2)   # prime a class instance with two initial values

# try updating v using getters and setters, then direct updates 

print( f'MyClass allows access to instance.v ({instance.get_v()}) via MyClass.get_v' ) 
instance.set_v( instance.get_v() + 2 )
print( f'MyClass allows updates to instance.v ({instance.get_v()}) via MyClass.set_v', end='\n\n' ) 

try:
  print( f"MyClass allows instance.v ({instance.v}) to be directly read" ) 
except LookupError as exception:
  print( f'MyClass disallows read access to instance.v: {make_printable(exception)}' ) 

try:
  instance.v += 2
  print( f'MyClass allows instance.v ({instance.v}) to be directly updated', end='\n\n' ) 
except LookupError as exception:
  print( f'MyClass disallows direct updates to instance.v: {make_printable(exception)}', end='\n\n' ) 

# try updating w using getters and setters, then direct updates 

print( f'MyClass allows access to instance.w ({instance.get_w()}) via MyClass.get_w' ) 
instance.set_w( instance.get_w() + 2 )
print( f'MyClass allows updates to instance.w ({instance.get_w()}) via MyClass.set_w', end='\n\n' ) 

try:
  print( f'MyClass allows instance.w ({instance.w}) to be directly read' ) 
except LookupError as exception:
  print( f'MyClass disallows read access to instance.w: { make_printable(exception)}' ) 

try:
  instance.w += 2
  print( f'MyClass allows instance.w ({instance.w}) to be directly updated' ) 
except LookupError as exception:
  print( f'MyClass disallows direct updates to instance.w: {make_printable(exception)}' )

##  10.6  Properties   <a name='Classes-Properties'></a>

The `property` functional allows an identifier to stand-in for a getter, setter, deleter, and/or docstring setter. `property` is invoked in one of three ways:
-  As a function with keywords.  A call like  
   &ensp;&ensp;&ensp;&ensp; `foo = property(fget=get_foo, fset=set_foo, fdel=del_foo, fdoc=foo_doc)`  
   in an instance *x* of a class allows client codes to use expressions like these:  
   &ensp;&ensp;&ensp;&ensp;  `print(x.foo)` &ensp;&ensp;&ensp;&ensp; in lieu of  print(*x.get_foo*())  
   &ensp;&ensp;&ensp;&ensp;  `x.foo` = 3   &ensp;&ensp;&ensp;&ensp; in lieu of  *x.set_foo*(3)  
   &ensp;&ensp;&ensp;&ensp;  `del x.foo`    &ensp;&ensp;&ensp;&ensp; in lieu of  *x.del_foo*()  
   &ensp;&ensp;&ensp;&ensp;  `foo.doc`      &ensp;&ensp;&ensp;&ensp; in lieu of  x.foo.\_\_doc\_\_  
   This use of keywords allows for the definition of any combination of these four methods.  <br><br>
-  As a function with four positional parameters: a getter, a setter,    a deleter, and a docstring, all of which default to `None`.  The following are some sample calls and their effects:
   - `foo = property(get_foo)`       &ensp;&ensp;&ensp;&ensp;   # defines a getter
   - `foo = property(None, set_foo)`  &ensp;&ensp;&ensp;&ensp;  # defines a setter
   - `foo = property(get_foo, None, None, foo_docstring)`  &ensp;&ensp;&ensp;&ensp;     # defines a getter and a docstring
   - `foo = property(get_foo, set_foo, del_foo, foo_docstring)` &ensp;&ensp;&ensp;&ensp; # defines all four methods  <br><br>
-  Using property decorators.  Here,
    -  an initial decorator, `@property`, is used to specify that a function--
       say, *foo*, is a getter for a property named foo:  
 `@property`  
 `def foo(self):`  
        &ensp;&ensp; """ `a docstring for foo, if one is wanted, goes here` """  
       &ensp;&ensp; `....`
    -  subsequent decorators with the property name appended to the function name then specify the property's setter and/or deleter, as desired.       The functions being decorated must also have the property's name:  e.g.,  
       `@foo.setter`  
       `def foo(self, value):`  
          `....`  
       `@foo.deleter(self):`  
          `....` 

When naming a property, use a name that differs from the names of *all*  of a class's instance variables. Using the same name for a property and an instance variable causes Python to repeatedly treat the name as a property rather than an attribute of `self`, leading to stack overflow and a program crash.

In [ ]:
# 10.6.a  using properties to specify and retrieve a circle's dimensions

class Circle(object):
  pi = 3.14159
  #
  def __init__(self, **kwargs):
    if len(set(['radius', 'diameter', 'circumference', 'area']) & set(kwargs.keys())) != 1:
       raise KeyError("constructor must be called with one keyword from 'radius', 'diameter', 'circumference', or 'area'")
    try:  self.radius = kwargs['radius']
    except:
      try:  self.diameter = kwargs['diameter']
      except:
        try:  self.circumference = kwargs['circumference']
        except:  self.area = kwargs['area']
  #
  # note that the radius methods are the only methods that 'know' that self.r stores persistent data
  def getradius(self):
    if not 'r' in dir(self):  raise UnboundLocalError("radius undefined")
    return self.r
  def setradius(self, r):  self.r = r
  def delradius(self):
    if 'r' in dir(self):   del self.r
  radius = property(getradius, setradius, delradius, 'circle radius')
  #
  def getdiameter(self):
    try:    return self.radius * 2
    except: raise UnboundLocalError("diameter undefined")
  def setdiameter(self, d):  self.radius = d / 2
  def deldiameter(self):     del self.radius
  diameter = property(getdiameter, setdiameter, deldiameter, 'circle diameter')
  #
  def getcircumference(self):
    try:    return 2 * Circle.pi * self.radius
    except: raise UnboundLocalError("circumference undefined")
  def setcircumference(self, c):  self.radius = c / (2 * Circle.pi)
  def delcircumference(self):     del self.radius
  circumference = property(getcircumference, setcircumference, delcircumference, 'circle circumference')
  #
  def getarea(self):
    try:    return Circle.pi * self.radius * self.radius
    except: raise UnboundLocalError("area undefined")
  def setarea(self, a):   self.radius = pow(a / Circle.pi, 0.5)
  def delarea(self):      del self.radius
  area = property(getarea, setarea, delarea, 'circle area')

#  show the four property-based views of a circle's attributes

c = Circle(radius=3)
message = 'a circle with radius {} has a diameter of {}, circumference of {}, and area of {}'
print( message.format( c.radius, c.diameter, c.circumference, c.area ), end='\n\n' )

#  show the effect of updating one property on the circle's other properties

c.area = 3
message = 'setting area to {} changes radius to {}, diameter to {}, and circumference to {}'
print( message.format( c.area, c.radius, c.diameter, c.circumference ), end='\n\n' )

#  show that deleting the property's underlying attribute causes other references to properties to fail

del c.circumference
print( 'after deleting the circle\'s circumference attribute,' )
print( '-  the circle\'s radius is', end='' )
try:    print( c.radius )
except: print( ' undefined' )
print( '-  the circle\'s diameter is', end='' )
try:    print( c.diameter )
except: print( ' undefined' )
print( '-  the circle\'s area is', end='' )
try:    print( c.area )
except: print( ' undefined' )

In [ ]:
# 10.6.b  the previous example, redone in part with properties

class Circle(object):
  pi = 3.14159
  #
  def __init__(self, **kwargs):
    if len(set(['radius', 'diameter', 'circumference', 'area']) & set(kwargs.keys())) != 1:
       raise KeyError("constructor must be called with one keyword from 'radius', 'diameter', 'circumference', or 'area'")
    try:  self.radius = kwargs['radius']
    except:
      try:  self.diameter = kwargs['diameter']
      except:
        try:  self.circumference = kwargs['circumference']
        except:  self.area = kwargs['area']
  #
  # note that the radius methods are the only methods that 'know' that self.r stores persistent data
  @property
  def radius(self):
    """circle radius"""
    if not 'r' in dir(self):  raise UnboundLocalError("radius undefined")
    return self.r
  @radius.setter
  def radius(self, r):   self.r = r
  @radius.deleter
  def radius(self):
    if 'r' in dir(self):  del self.r
  #
  @property
  def diameter(self):
    """circle diameter"""
    try:       return self.radius * 2
    except:    raise UnboundLocalError("diameter undefined")
  @diameter.setter
  def diameter(self, d):  self.radius = d / 2
  @diameter.deleter
  def diameter(self):     del self.radius
  #
  @property
  def circumference(self):
    """circle circumference"""
    try:       return 2 * Circle.pi * self.radius
    except:    raise UnboundLocalError("circumference undefined")
  @circumference.setter
  def circumference(self, c):  self.radius = c / (2 * Circle.pi)
  @circumference.deleter
  def circumference(self):     del self.radius
  #
  @property
  def area(self):
    """circle area"""
    try:       return Circle.pi * self.radius * self.radius
    except:    raise UnboundLocalError("area undefined")
  @area.setter
  def area(self, a):   self.radius = pow(a / Circle.pi, 0.5)
  @area.deleter
  def area(self):      del self.radius

#  show the four property-based views of a circle's attributes

c = Circle(radius=3)
message = 'a circle with radius {} has a diameter of {}, circumference of {}, and area of {}'
print( message.format( c.radius, c.diameter, c.circumference, c.area ), end='\n\n' )

## 10.7  Managing polymorphism  <a name='Classes-Managing-Polymorphism'></a>

The use of shorthand names for methods can force a language's run-time system to determine what method a name is referencing.  Consider the following scenario:
-  an instance of a class *class_A* invokes a method *m_1* defined in one of *A*'s parent classes, *parent_A*
-  *parent_A.m_1*, in turn, invokes a second method *m_2*, as *m_2*.
-  *m_2* is defined in two places:
   -  one in parent_A, *parent_A.m_2*
   -  one in class_A,  *class_A.m_2*

If the run-time system invokes *class_A.m_2* instead of *parent_A.m_2*, method *m_2* is said to be *virtual*. The term "virtual" is a confusing name for a policy that "simply" says the following:
- When confronted with an ambiguous reference to two methods with a common name, use the method in the caller.

Treating methods as virtual by default is regarded as a best practice in OO language design.  In Python, virtualization is a natural consequence of the use of `self` to qualify method names.

To avoid virtualization, invoke an instance method with an explicit reference to its container class.

In [ ]:
# 10.7   using virtual and non-virtual method calls to access class member data

class MyClass:
  def __init__(self, iv_1, iv_2):
    self.set_instance_value_1( iv_1 )              # - - - a virtual method call - - -
    MyClass.set_instance_value_2( self, iv_2 )     # - - - a non-virtual method call - - -
  #
  def set_instance_value_1(self, v):    self.instance_value_1 = 'set from MyClass: ' + str(v)
  def get_instance_value_1(self):       return self.instance_value_1
  #
  def set_instance_value_2(self, v):    self.instance_value_2 = 'set from MyClass: ' + str(v)
  def get_instance_value_2(self):       return self.instance_value_2

class MySubclass(MyClass):
  # important: MySubclass is inheriting and invoking MyClass's init method
  #
  def set_instance_value_1(self, v):    self.instance_value_1 = 'set from MySubclass: ' + str(v)
  def get_instance_value_1(self):       return self.instance_value_1
  #
  def set_instance_value_2(self, v):    self.instance_value_2 = 'set from MySubclass: ' + str(v)
  def get_instance_value_2(self):       return self.instance_value_2

# confirm that the initialization logic works as claimed

mysubclass_instance = MySubclass( 'first', 'second' )
print( 'effect of making a virtual call to set instance_value_1: ',     mysubclass_instance.get_instance_value_1() )
print( 'effect of making a non-virtual call to set instance_value_2: ', mysubclass_instance.get_instance_value_2() )

## 10.8  Multiple inheritance <a name='Classes-Multiple-Inheritance'></a>

### 10.8.1  About multiple inheritance <a name='Classes-Multiple-Inheritance-About'></a>

Python supports *multiple inheritance*: the inheritance of attributes from two or more superclasses. One typical use of multiple inheritance is adding shared content to two or more existing classes via *mixins*:  classes that typically
-  define a common set of definitions or
-  implement one common, complex action

Some people argue than using mixins to vary or enhance a class's behavior is less maintainable than using *dependency injection* to achieve this goal: i.e., passing an object with callable attributes as a parameter. Still, multiple inheritance can sometimes be an elegant way to define families of related objects,  including objects that represent various types of files and sockets: e.g., 
-  start with a class that defines basic socket support
-  over top of this, layer
    -  protocol support, in the form of either TCP or UDP protocol support
    -  role support, in the form of either client or server support
-  finally, combining these four possibilities yields
    - TCP client sockets
    - TCP server sockets
    - UDP client sockets
    - UDP server sockets

### 10.8.2  Multiple inheritance in Python <a name='Classes-Multiple-Inheritance-In-Python'><a/>

To specify that a class inherits from two or more classes, list that class's superclasses in its header, in order of priority. Python uses this ordering to resolve references: if a given name is defined in two or more of a class's base classes, Python will use the definition from the leftmost class in this list.

A class's primary superclass is recorded in that class's `__class__.__base__` attribute. The classes from which a given class inherits can be identified by invoking *any* (!!) class's `__class__.mro` method (short for "method resolution order") with the class of interest as its argument. Similarly, a class's subclasses can be identified by invoking *any* (!) class's `__class__.__subclasses__` method on the class of interest.

In [ ]:
# 10.8.2 illustrating multiple inheritance and the resolution of inheritance conflicts *** ***

class MyMainClass:
  def set_instance_value_1(self, v):    self.instance_value_1 = 'set from MyClass: ' + str(v)
  def get_instance_value_1(self):       return self.instance_value_1

class MyMixinClass_1:
  def set_instance_value_2(self, v):    self.instance_value_2 = 'set from MyMixinClass_1: ' + str(v)
  def get_instance_value_2(self):       return self.instance_value_2
  #
  def set_instance_value_4(self, v):    self.instance_value_4 = 'set from MyMixinClass_1: ' + str(v)
  def get_instance_value_4(self):       return self.instance_value_4

class MyMixinClass_2:
  def set_instance_value_3(self, v):    self.instance_value_3 = 'set from MyMixinClass_2: ' + str(v)
  def get_instance_value_3(self):       return self.instance_value_3
  #
  def set_instance_value_4(self, v):    self.instance_value_4 = 'set from MyMixinClass_2: ' + str(v)
  def get_instance_value_4(self):       return self.instance_value_4

class MySubclass(MyMainClass, MyMixinClass_1, MyMixinClass_2):  pass

# *** *** confirming the class hierarchy *** ***

print( 'MyMainClass\'s inheritance resolution order is ', object.__class__.mro(MyMainClass)[1:] )
print( 'MyMainClass\'s primary superclass is ', MyMainClass.__class__.__base__)
subclasses = object.__class__.__subclasses__(MyMainClass)
if subclasses:
  print( 'MyMainClass\'s subclasses are ', subclasses )
else:
  print( 'MyMainClass has no subclasses' )
print( '----' )

print( 'MyMixinClass_1\'s inheritance resolution order is ', object.__class__.mro(MyMixinClass_1)[1:] )
print( 'MyMixinClass_1\'s primary superclass is ', MyMixinClass_1.__class__.__base__)
subclasses = object.__class__.__subclasses__(MyMixinClass_1)
if subclasses:
  print( 'MyMixinClass_1\'s subclasses are ', subclasses )
else:
  print( 'MyMixinClass_1 has no subclasses' )
print( '----' )

print( 'MyMixinClass_2\'s inheritance resolution order is ', object.__class__.mro(MyMixinClass_2)[1:] )
print( 'MyMixinClass_2\'s primary superclass is ', MyMixinClass_2.__class__.__base__)
subclasses = object.__class__.__subclasses__(MyMixinClass_2)
if subclasses:
  print( 'MyMixinClass_2\'s subclasses are ', subclasses )
else:
  print( 'MyMixinClass_2 has no subclasses' )
print( '----' )

print( 'MySubclass\'s inheritance resolution order is ', object.__class__.mro(MySubclass)[1:] )
print( 'MySubclass\'s primary superclass is ', MySubclass.__class__.__base__)
subclasses = object.__class__.__subclasses__(MySubclass)
if subclasses:
  print( 'MySubclass\'s subclasses are ', subclasses )
else:
  print( 'MySubclass has no subclasses' )
print( '----' )

# *** *** invoking final, mixin-based class methods *** ***

my_subclass_instance = MySubclass()
print( 'Setting subclass value 1 to \'one\', 2 to \'two\', 3 to \'three\', and 4 to \'four\' ' )
my_subclass_instance.set_instance_value_1('one')
my_subclass_instance.set_instance_value_2('two')
my_subclass_instance.set_instance_value_3('three')
my_subclass_instance.set_instance_value_4('four')
print( )

print( 'Retrieving the four values.  Note that the two mixin classes each define a set method for value 4' )
print( 'value 1 is ', my_subclass_instance.get_instance_value_1() )
print( 'value 2 is ', my_subclass_instance.get_instance_value_2() )
print( 'value 3 is ', my_subclass_instance.get_instance_value_3() )
print( 'value 4 is ', my_subclass_instance.get_instance_value_4() )

<span style='color:blue'>&#128073;&ensp;&ensp;**Exercise 10.8.2.`**

</span><span style='color:navy'>In the following code cell, show the method resolution order for `int` and `object`.</span>

### 10.8.3  Accessing methods hidden by other methods <a name='Classes-Multiple-Inheritance-Accessing_Hidden-Methods'></a>

An instance method *m* in a class *C* that is hidden by another method of the same name can be invoked in one of two ways:
-  by qualifying it with the name of the class that holds the version of *m* to invoke, and passing *self* as the invocation's    first parameter:  e.g., by invoking *C.m( self, ... )*, where "..." denotes *m*'s parameters.
-  by invoking *super().m( .... )*, where "..." denotes *m*'s parameters.  This strategy assumes that class *C* is    the first class in the calling class's method resolution order with a method named *m*.

This need to call a superclass method with the same name as a subclass method arises commonly for class constructors, where there's a need to invoke the superclass's `__init__` method to initialize superclass for the subclass's use.

In [ ]:
# 10.8.3.a  accessing hidden methods - hidden method in immediate superclass
# referencing MyClass.set_value_1 in MyClass.__init__ prevents call to MySubclass.set_value_1
# referencing self.set_value_2 in MyClass.__init__ invokes MySubclass.set_value_2, due to virtualization

class MyClass:
  def __init__(self, **kwds ):
    MyClass.set_value_1( self, **kwds )
    self.set_value_2( **kwds )
  #
  def set_value_1(self, **kwds ):
    self.value_1 = ('value_1, as set by MyClass', kwds.get('value_1', None))
  #
  def set_value_2(self, **kwds ):
    self.value_2 = ('value_2, as set by MyClass', kwds.get('value_2', None))

class MySubclass(MyClass):
  def __init__(self, **kwds ):
    super().__init__( **kwds )
  #
  def set_value_1(self, **kwds ):
    self.value_1 = ('value_1, as set by MySubclass', kwds.get('value_1', None))
  #
  def set_value_2(self, **kwds ):
    self.value_2 = ('value_2, as set by MySubclass', kwds.get('value_2', None))
  #
  def get_values(self):
    return (self.value_1, self.value_2)

# - - - - - - confirming the methods' operation - - - - - -

my_subclass_instance = MySubclass(value_1=11, value_2=22)
print( 'Invoking MySubclass(value_1=11, value_2=22) yields', my_subclass_instance.get_values() )

In [ ]:
# 10.8.3.b  accessing hidden methods - hidden method in hidden superclass

class MyMainClass:
  def set_value_1(self, v):    self.value_1 = 'set from MyMainClass: ' + str(v)
  def get_value_1(self):       return self.value_1
  #
  def set_value_2(self, v):    self.value_2 = 'set from MyMainClass: ' + str(v)
  def get_value_2(self):       return self.value_2

class MyMixinClass:
  def set_value_2(self, v):    self.value_2 = 'set from MyMixinClass: ' + str(v)
  def get_value_2(self):       return self.value_2

class MySubclass(MyMainClass, MyMixinClass):
  def __init__(self, v1, v2 ):
    MyMainClass.set_value_1( self, v1 )
    MyMixinClass.set_value_2( self, v2 )
  #
  def get_values(self):
    return (self.value_1, self.value_2)

my_subclass_instance = MySubclass(11, 22)
print( 'Invoking MySubclass(11, 22) yields', my_subclass_instance.get_values() )